# Land Surface Tempeature in Kochi and Trivandrum

Land surface temperature (LST) is defined as how hot the surface would feel to the touch and which cools and warms more quickly than the air above it. In this case, “surface” is the nearest solid on the satellite’s point of view—which can include actual land, concrete, or canopy, for example. Many modern satellites, such as MODIS, from NASA, measure LST. 

Land Surface Temperature (LST), can be a proxy, albeit not the most accurate one, for relatuve differences in air temperature. While LST indicators cannot be used by themeselves, they allow for a highly granular satellite image which can be used to compare with coarser air temperature readings in a city. Especially for city level governments LST shows the difference in temperature between neighborhoods. This data can also be obtained free of cost. 

## Data

## LANDSAT

The data for Land Surface Temperature was taken from extracted using Google Earth Engine, from the [LANDSAT 8 Level 2](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2) satellite imagery, courtesy of  [U.S. Geological Survey](https://www.usgs.gov/landsat-missions/landsat-collection-2-level-2-science-products). These images contain 5 visible and near-infrared (VNIR) bands and 2 short-wave infrared (SWIR) bands processed to orthorectified surface reflectance, and one thermal infrared (TIR) band processed to orthorectified surface temperature. 

## High Resolution Population Density Maps

The temperature data is overlayed with population data, for elderly population who tend to be more vulnerable to heatwaves. The population statistics are obtained from Meta's [High Resolution Population Density Maps](https://ciesin.columbia.edu/data/hrsl/). 

In partnership with the Center for International Earth Science Information Network (CIESIN) at Columbia University, Meta uses state-of-the-art computer vision techniques to identify buildings from publicly accessible mapping services to create these population datasets. These maps are available at 30-meter resolution. These maps aren’t built using Facebook data and instead rely on applying machine vision AI to satellite imagery and census information (latest available). 

## Methodology

The LST dataset is best used in conjuction with other data from ERA-5 and any satellite imagery available to the government of Kochi and Trivandrum (i.e., ISRO). In this analysis, we overlayed the LST maps with elderly population aggregated to H3 grids at resolution 9 i.e., 0.105 sq.km. area. 

## Interactive Heatmap of Kochi



In [54]:
import asyncio
import json
import os
from typing import Optional

import geopandas
import h3
import pandas as pd
import shapely
from aiohttp import ClientSession
from shapely.geometry import Polygon

In [4]:
kochi = geopandas.read_file('../data/shapefiles/kochi.shp')
trivandrum = geopandas.read_file('../data/shapefiles/trivandrum.shp')

In [55]:
import geopandas
def get_h3_tessellation(
    gdf: geopandas.GeoDataFrame, name="shapeName", resolution=10
) -> geopandas.GeoDataFrame:
    mapper = dict()
    tiles = set()

    # TODO: vectorize, if possible
    for idx, row in gdf.iterrows():
        geometry = row["geometry"]

        match geometry.geom_type:
            case "Polygon":
                hex_ids = h3.polyfill(
                    shapely.geometry.mapping(geometry),
                    resolution,
                    geo_json_conformant=True,
                )

                tiles = tiles.union(set(hex_ids))
                mapper.update([(hex_id, row[name]) for hex_id in hex_ids])

            case "MultiPolygon":
                for x in geometry.geoms:
                    hex_ids = h3.polyfill(
                        shapely.geometry.mapping(x),
                        resolution,
                        geo_json_conformant=True,
                    )

                    tiles = tiles.union(set(hex_ids))
                    mapper.update([(hex_id, row[name]) for hex_id in hex_ids])
            case _:
                raise (Exception)

    tessellation = geopandas.GeoDataFrame(
        data=tiles,
        geometry=[Polygon(h3.h3_to_geo_boundary(idx, True)) for idx in tiles],
        columns=["hex_id"],
        crs="EPSG:4326",
    )

    return tessellation

In [107]:
import shapely
kochi_h3 = get_h3_tessellation(kochi, name="NAME", resolution=9)
kochi_h3.to_file('../data/shapefiles/kochi_h3.shp', format='ESRI Shapefile')

In [149]:
trivandrum_h3 = get_h3_tessellation(trivandrum, name = 'NAME', resolution =9)
trivandrum_h3.to_file('../data/shapefiles/trivandrum_h3.shp', format='ESRI Shapefile')

<iframe width="100%" height="500px" src="https://studio.foursquare.com/map/public/40faf349-0033-4afd-826c-09c62a447443/embed" frameborder="0" allowfullscreen></iframe>

### Observations

- It can be seen already that the most densely populated regions of Kochi are the hottest
- The river flowing through the middle of the city has the lowest recorded temperature which is in alignment with expectations. 
- The Edakochi area has multiple hospitals and points of interest which are in relatively hotter parts of the city

## Interactive Heatmap of Trivandrum

<iframe width="100%" height="500px" src="https://studio.foursquare.com/map/public/af2ad727-78e6-4d7f-97ee-de47b23a97de/embed" frameborder="0" allowfullscreen></iframe>

### Observations

- It can already be seen that Trivandrum is hotter than Kochi. It could be because there is no waterbody flowing through the city
- Although the coastal region of Eanchakkal is one of the hottest, the lederly population seems to be consentrated in Chala and Thampanoor. 